In [67]:
import pandas as pd
from qdrant_client import QdrantClient, models
# from FlagEmbedding import BGEM3FlagModel
from openai import OpenAI
import os
import ast
import re
import json
import tqdm
from typing import List, Dict, Any, Iterable
from dotenv import load_dotenv
from utils import *
import uuid
load_dotenv()

True

# CREATE VECTOR

In [36]:
import hashlib

def generate_point_id(cid:  int, chunk_text: str) -> str:
    hash_obj = hashlib.sha1(f"{cid}-{chunk_text}".encode('utf-8')).hexdigest()
    return str(uuid.UUID(hash_obj[:32]))

In [37]:
def build_clients():
    oa = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
    qdrant = QdrantClient(
        url=os.environ.get("QDRANT_URL", "http://localhost:6333"),  # None nếu local
        timeout=60,
    )
    return oa, qdrant

In [38]:
def embed_texts(client_oa: OpenAI, texts: List[str], model: str) -> List[List[float]]:
    """Embed 1 danh sách texts, trả về list vectors theo đúng thứ tự."""
    resp = client_oa.embeddings.create(model=model, input=texts)
    return [item.embedding for item in resp.data]

In [39]:

def create_fixed_chunks(text, max_word_count=400):
    sentences = re.split(r'(?<=[.!?]) +', text)
    chunks, current_chunk, word_count = [], "", 0
    for sentence in sentences:
        wc = len(sentence.split())
        if word_count + wc > max_word_count:
            if current_chunk:
                chunks.append(current_chunk.strip())
            current_chunk, word_count = sentence, wc
        else:
            current_chunk += " " + sentence if current_chunk else sentence
            word_count += wc
    if current_chunk:
        chunks.append(current_chunk.strip())
    return chunks

In [70]:
from qdrant_client.models import VectorParams, Distance, PointStruct
from tqdm import tqdm

def index_corpus_to_qdrant(data, model, vector_size, collection_name: str):
    
    oa, qdrant = build_clients()
    
    qdrant.recreate_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(size=vector_size, distance=Distance.COSINE),
    )
    # Build points
    points = []
    for row in tqdm(data.itertuples(), total=len(data)):
        chunks = create_fixed_chunks(row.text)
        dense_vectors = embed_texts(oa, chunks, model)
        for i, (chunk, dense_vector) in enumerate(zip(chunks, dense_vectors)):
            point_id = generate_point_id(row.cid, chunk)
            payload = {
                "cid": int(row.cid),
                "chunk_index": i,
                "text": chunk
            }
            
            points.append(PointStruct(
                id=point_id, 
                vector=dense_vector,   
                payload=payload))
        if len(points) > 50:
            # Upsert to Qdrant
            print(f"Uploading {len(points)} vectors to Qdrant...")
            qdrant.upsert(collection_name=collection_name, points=points)
            points = []
            print("✅ Done uploading batch.")
            
        # Upsert remaining points
    if points:
        print(f"Uploading remaining {len(points)} vectors to Qdrant...")
        qdrant.upsert(collection_name=collection_name, points=points)
        print("✅ Done uploading final batch.")

In [61]:
MODEL = "text-embedding-3-small"  # hoặc "text-embedding-3-large"
MODEL_DIM = 1536
COLLECTION = "law_corpus_openai"         # tên collection trong Qdrant
data =pd.read_csv(r"D:\Data\Legal-Retrieval\data\data_corpus\corpus.csv")
df_shuffled = data.sample(frac=1, random_state=42).reset_index(drop=True)

In [74]:
df_save = df_shuffled[:2000]  # Tăng từ 1000 lên 2000 mẫu
print(f"Số lượng documents: {len(df_save)}")

Số lượng documents: 2000


In [ ]:
index_corpus_to_qdrant(df_save,MODEL,MODEL_DIM,COLLECTION)

In [ ]:

# oa, qdrant = build_clients()
# vector_size = MODEL_DIM
# collection_name = COLLECTION
# model = MODEL
# qdrant.recreate_collection(
#     collection_name=collection_name,
#     vectors_config=VectorParams(size=vector_size, distance=Distance.COSINE),
# )
# # Build points
# points = []
# for row in tqdm(df_save.itertuples(), total=len(df_save)):
#     chunks = create_fixed_chunks(row.text)
#     dense_vectors = embed_texts(oa,chunks,model)
#     for i, (chunk,dense_vector) in enumerate(zip(chunks,dense_vectors)):
#         point_id = generate_point_id(row.cid, chunk)
#         payload = {
#             "cid": int(row.cid),
#             "chunk_index": i,
#             "text": chunk
#         }
        
#         points.append(PointStruct(
#             id=point_id, 
#             vector=dense_vector,   
#             # "sparse": sparse_vector  
#             payload=payload))
#     if len(points) > 50:
#         # Upsert to Qdrant
#         # print(f"Uploading {len(points)} vectors to Qdrant...")
#         qdrant.upsert(collection_name=collection_name, points=points)
#         points = []
#         # print("✅ Done uploading batch.")
        
#     # Upsert remaining points
# if points:
#     print(f"Uploading remaining {len(points)} vectors to Qdrant...")
#     qdrant.upsert(collection_name=collection_name, points=points)
#     print("✅ Done uploading final batch.")

C:\Users\Admin\AppData\Local\Temp\ipykernel_31000\3725064985.py:3: UserWarning: Qdrant client version 1.15.0 is incompatible with server version 1.8.4. Major versions should match and minor version difference must not exceed 1. Set check_compatibility=False to skip version check.
  qdrant = QdrantClient(
C:\Users\Admin\AppData\Local\Temp\ipykernel_31000\3967376385.py:5: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(
100%|██████████| 4/4 [00:02<00:00,  1.64it/s]

Uploading remaining 6 vectors to Qdrant...
✅ Done uploading final batch.


In [76]:
# Test search trực tiếp trên collection
oa, qdrant = build_clients()

# Test query
test_query = "luật hợp đồng lao động"
print(f"Testing query: {test_query}")

# Get embedding
vector = embed_texts(oa, [test_query], MODEL)[0]
print(f"Vector dimension: {len(vector)}")

# Search
results = qdrant.search(
    collection_name=COLLECTION,
    query_vector=vector,
    limit=5,
    with_payload=True
)

print(f"\nFound {len(results)} results:")
for i, result in enumerate(results):
    print(f"\n{i+1}. Score: {result.score:.4f}")
    print(f"   CID: {result.payload.get('cid')}")
    print(f"   Text: {result.payload.get('text')[:150]}...")

C:\Users\Admin\AppData\Local\Temp\ipykernel_31000\3725064985.py:3: UserWarning: Qdrant client version 1.15.0 is incompatible with server version 1.8.4. Major versions should match and minor version difference must not exceed 1. Set check_compatibility=False to skip version check.
  qdrant = QdrantClient(


Testing query: luật hợp đồng lao động
Vector dimension: 1536

Found 5 results:

1. Score: 0.5847
   CID: 30328
   Text: Thông tư này quy định về mức trần tiền ký quỹ và thị trường lao động mà doanh nghiệp hoạt động dịch vụ đưa người lao động Việt Nam đi làm việc ở nước ...

2. Score: 0.5577
   CID: 636702
   Text: Mục 4. HỢP ĐỒNG LAO ĐỘNG VÔ HIỆU
Điều 49. Hợp đồng lao động vô hiệu
1. Hợp đồng lao động vô hiệu toàn bộ trong trường hợp sau đây:
a) Toàn bộ nội dung...

3. Score: 0.5565
   CID: 28255
   Text: 1. Nghĩa vụ của người lao động khi đơn phương chấm dứt hợp đồng lao động không đúng quy định tại Điều 11 Nghị định số 27/2014/NĐ-CP:
a) Không được trợ...

4. Score: 0.5521
   CID: 65552
   Text: Quyền đơn phương chấm dứt hợp đồng lao động của người sử dụng lao động
1. Người sử dụng lao động có quyền đơn phương chấm dứt hợp đồng lao động trong ...

5. Score: 0.5494
   CID: 122382
   Text: "Điều 7. Các hành vi bị nghiêm cấm trong lĩnh vực người lao động Việt Nam đi làm việc ở nước ngoài

C:\Users\Admin\AppData\Local\Temp\ipykernel_31000\3489158105.py:13: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = qdrant.search(


In [77]:
# Test nhiều queries để kiểm tra khả năng retrieval
oa, qdrant = build_clients()

test_queries = [
    "luật hợp đồng lao động",
    "thời gian làm việc", 
    "bảo hiểm xã hội",
    "nghỉ phép năm",
    "chấm dứt hợp đồng",
    "What is labor law?",  # English query
    "vi phạm luật",
    "trách nhiệm người sử dụng lao động"
]

print("🔍 Testing Multiple Queries on Local Collection:")
print("=" * 60)

for i, query in enumerate(test_queries, 1):
    print(f"\n{i}. Query: '{query}'")
    
    # Get embedding
    try:
        vector = embed_texts(oa, [query], MODEL)[0]
        
        # Search
        results = qdrant.search(
            collection_name=COLLECTION,
            query_vector=vector,
            limit=3,
            with_payload=True
        )
        
        if results:
            print(f"   ✅ Found {len(results)} results (scores: {[f'{r.score:.3f}' for r in results]})")
            best_result = results[0]
            print(f"   📄 Best match: {best_result.payload.get('text')[:100]}...")
        else:
            print(f"   ❌ No results found")
            
    except Exception as e:
        print(f"   ❌ Error: {e}")

print(f"\n📊 Collection Stats:")
collection_info = qdrant.get_collection(COLLECTION)
print(f"   Vectors: {collection_info.vectors_count}")
print(f"   Points: {collection_info.points_count}")

C:\Users\Admin\AppData\Local\Temp\ipykernel_31000\3725064985.py:3: UserWarning: Qdrant client version 1.15.0 is incompatible with server version 1.8.4. Major versions should match and minor version difference must not exceed 1. Set check_compatibility=False to skip version check.
  qdrant = QdrantClient(


🔍 Testing Multiple Queries on Local Collection:

1. Query: 'luật hợp đồng lao động'


C:\Users\Admin\AppData\Local\Temp\ipykernel_31000\1341456841.py:26: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = qdrant.search(


   ✅ Found 3 results (scores: ['0.585', '0.558', '0.556'])
   📄 Best match: Thông tư này quy định về mức trần tiền ký quỹ và thị trường lao động mà doanh nghiệp hoạt động dịch ...

2. Query: 'thời gian làm việc'
   ✅ Found 3 results (scores: ['0.682', '0.550', '0.490'])
   📄 Best match: Thời gian làm việc
1. Chấp hành nghiêm quy định về thời gian làm việc của Nhà nước, của Ngành, của c...

3. Query: 'bảo hiểm xã hội'
   ✅ Found 3 results (scores: ['0.682', '0.550', '0.490'])
   📄 Best match: Thời gian làm việc
1. Chấp hành nghiêm quy định về thời gian làm việc của Nhà nước, của Ngành, của c...

3. Query: 'bảo hiểm xã hội'
   ✅ Found 3 results (scores: ['0.541', '0.525', '0.519'])
   📄 Best match: Nguồn vốn thành lập tổ chức bảo hiểm tương hỗ bao gồm:
1. Đóng góp của các thành viên sáng lập.
2. T...

4. Query: 'nghỉ phép năm'
   ✅ Found 3 results (scores: ['0.541', '0.525', '0.519'])
   📄 Best match: Nguồn vốn thành lập tổ chức bảo hiểm tương hỗ bao gồm:
1. Đóng góp của các thành viên s

In [ ]:
# class QuestionInference:
#     def __init__(self, csv_path, save_pair_path, qdrant_search):
#         self.csv_path = csv_path
#         self.save_pair_path = save_pair_path
#         self.qdrant_search = qdrant_search
    
#     def load_questions(self):
#         """Load questions and question_ids from CSV file"""
#         self.questions = pd.read_csv(self.csv_path)
    
#     def infer_and_save(self):
#         """Infer each question and save results to a .txt file"""
#         file_name = "data_round1"
#         with open(os.path.join(self.save_pair_path, file_name + '.json'), 'w') as output_file:
#             for row in tqdm.tqdm(self.questions.itertuples(index=False)):
#                 question = row.question
#                 list_id = convert_to_list(row.cid)
#                 list_context = convert_str_to_list(row.context)
#                 # create_data for bge
#                 save_dict = {}
#                 save_dict["query"] = question
#                 save_dict["pos"] = []
#                 save_dict["neg"] = []
#                 for context in list_context:
#                     chunk_context = split_text_keeping_sentences(text=context, max_word_count=400)
#                     save_dict["pos"] += chunk_context

#                 results = self.qdrant_search.search(query_text=question, limit=25)
#                 for result in results.points:
#                     infor_id = int(result.payload["infor_id"])
#                     if infor_id in list_id:
#                         continue
#                     else:
#                         text = result.payload["text"]
#                         save_dict["neg"].append(text)

#                 output_file.write(json.dumps(save_dict,ensure_ascii=False) + '\n')
                
                

In [ ]:
# qdrant_search = QdrantSearch_bge(
#     host="http://localhost:6333",
#     collection_name="law_with_bge_round1",
#     model_name="BAAI/bge-m3",
#     use_fp16=True
# )

In [ ]:
# questions = pd.read_csv(r"D:\Data\Legal-Retrieval\data\train.csv")
# questions